In [ ]:
#default_exp notebook.showdoc

In [ ]:
# export
from fastai_local.core import *
from fastai_local.imports import *
from fastai_local.data.pipeline import *
from fastai_local.data.external import *
from fastai_local.test import *
from fastai_local.notebook.export import *
import inspect,enum
from IPython.display import Markdown,display

# Show doc
> Functions to show the doc cells in notebooks

In [ ]:
test_cases = [
    listify,    #Basic func
    Pipeline,   #Basic class
    ConfigKey,  #Enum
    compose,    #Func with star args and type annotation
    untar_data, #Func with defaults
    add_docs    #Func with kwargs
]

## Gather the information

In [ ]:
def fn_name(ft):
    "Get the name of `ft`"
    if hasattr(ft, '__name__'):       return ft.__name__
    elif getattr(ft, '_name', False): return ft._name
    else:                             return str(ft).split('.')[-1]

In [ ]:
test_eq([fn_name(o) for o in test_cases], 
        ['listify', 'Pipeline', 'ConfigKey', 'compose', 'untar_data', 'add_docs'])

In [ ]:
def get_anchor(fn):
    "Anchor name associated to `fn`"
    if hasattr(fn,'__qualname__'): return fn.__qualname__
    if inspect.ismethod(fn): return fn_name(fn.__self__) + '.' + fn_name(fn)
    return fn_name(fn)

In [ ]:
test_eq([get_anchor(o) for o in test_cases], 
        ['listify', 'Pipeline', 'ConfigKey', 'compose', 'untar_data', 'add_docs'])
test_eq(get_anchor(Pipeline.composed), 'Pipeline.composed')

The inspect module lets us know quickly if an object is a function or a class but it doesn't distinguish classes and enums.

In [ ]:
def is_enum(cls): 
    "Check if `cls` is an enum or another type of class"
    return cls.__class__ == enum.Enum or cls.__class__ == enum.EnumMeta

In [ ]:
assert is_enum(ConfigKey)
assert not is_enum(Pipeline)

In [ ]:
def _get_mod_name(ft): 
    "Return the name of the module where `ft` is defined"
    return inspect.getmodule(ft).__name__

In [ ]:
test_eq(_get_mod_name(Pipeline), 'fastai_local.data.pipeline')
test_eq(_get_mod_name(untar_data), 'fastai_local.data.external')
test_eq(_get_mod_name(ConfigKey), 'fastai_local.data.external')

### Links

In [ ]:
#hide
#Tricking jupyter notebook to have a __file__ attribute. All _file_ will be replaced by __file__
_file_ = Path('fastai_local').absolute()/'notebook'/'show_doc.py'

In [ ]:
def _get_pytorch_index():
    if not (Path(_file_).parent/'index_pytorch.txt').exists(): return {}
    return json.load(open(Path(_file_).parent/'index_pytorch.txt', 'r'))

def add_pytorch_index(func_name, url):
    "Add `func_name` in the PyTorch index for automatic links."
    index = _get_pytorch_index()
    if not url.startswith("https://pytorch.org/docs/stable/"):
        url = "https://pytorch.org/docs/stable/" + url
    index[func_name] = url
    json.dump(index, open(Path(_file_).parent/'index_pytorch.txt', 'w'), indent=2)

`url` can be the full url or just the part after `https://pytorch.org/docs/stable/`, see the example below.

In [ ]:
#hide
ind,ind_bak = Path(_file_).parent/'index_pytorch.txt',Path(_file_).parent/'index_pytorch.bak'
if ind.exists(): shutil.move(ind, ind_bak)
test_eq(_get_pytorch_index(), {})
add_pytorch_index('Tensor', 'tensors.html#torch-tensor')
test_eq(_get_pytorch_index(), {'Tensor':'https://pytorch.org/docs/stable/tensors.html#torch-tensor'})
if ind_bak.exists(): shutil.move(ind_bak, ind)

In [ ]:
add_pytorch_index('Tensor', 'tensors.html#torch-tensor')
add_pytorch_index('device', 'tensor_attributes.html#torch-device')
add_pytorch_index('DataLoader', 'data.html#torch.utils.data.DataLoader')

In [ ]:
def is_fastai_module(name):
    "Test if `name` is a fastai module."
    return (Path(_file_).parent.parent/f"{name}.py").exists()

In [ ]:
#Might change once the library is renamed fastai.
def _is_fastai_class(ft): return belongs_to_module(ft, 'fastai_source')
def _strip_fastai(s): return re.sub(r'^fastai_local\.', '', s)
FASTAI_DOCS = '' #TODO: change when live

In [ ]:
def doc_link(name, include_bt:bool=True):
    "Create link to documentation for `name`."
    try_fastai = source_nb(name, is_name=True)
    cname = f'`{name}`' if include_bt else name
    if try_fastai:
        page = '_'.join(try_fastai.split('_')[1:]).replace('.ipynb', '.html')
        return f'[{cname}]({FASTAI_DOCS}/{page}#{name})'
    try_pytorch = _get_pytorch_index().get(name, None)
    if try_pytorch: return f'[{cname}]({try_pytorch})'
    return name

In [ ]:
test_eq(doc_link('Pipeline'), f'[`Pipeline`]({FASTAI_DOCS}/data_pipeline.html#Pipeline)')
test_eq(doc_link('Transform.create'), 
        f'[`Transform.create`]({FASTAI_DOCS}/data_pipeline.html#Transform.create)')
test_eq(doc_link('Tensor'), '[`Tensor`](https://pytorch.org/docs/stable/tensors.html#torch-tensor)')
test_eq(doc_link('Tenso'), 'Tenso')

In [ ]:
import importlib

In [ ]:
mod = importlib.import_module('notebook', 'fastai_local')

In [ ]:
mod

<module 'notebook' from '/home/ubuntu/anaconda3/lib/python3.7/site-packages/notebook/__init__.py'>

In [ ]:
import pkgutil

In [ ]:
[m for m in pkgutil.iter_modules('fastai_local')]

ValueError: path must be None or list of paths to look for modules in

In [ ]:
importlib.

In [ ]:
dir(mod)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'core',
 'data',
 'imports',
 'notebook',
 'test']

In [ ]:
mod.notebook

<module 'fastai_local.notebook' (namespace)>

In [ ]:
def add_doc_links(text):
    "Try to find doc link for any item between backticks in `text`."
    

In [ ]:
inspect._VAR_POSITIONAL

<_ParameterKind.VAR_POSITIONAL: 2>

In [ ]:
arg_prefixes = {inspect._VAR_POSITIONAL: '\*', inspect._VAR_KEYWORD:'\*\*'}
def code_esc(s): return f'`{s}`'

In [ ]:
def is_enum(cls): return cls == enum.Enum or cls == enum.EnumMeta

In [ ]:
def link_type(arg_type, arg_name=None, include_bt:bool=True):
    "Create link to documentation."
    arg_name = arg_name or fn_name(arg_type)
    if include_bt: arg_name = code_esc(arg_name)
    return arg_name
    if belongs_to_module(arg_type, 'torch') and ('Tensor' not in arg_name): return f'[{arg_name}]({get_pytorch_link(arg_type)})'
    if is_fastai_class(arg_type): return f'[{arg_name}]({get_fn_link(arg_type)})'
    return arg_name

In [ ]:
def type_repr(t):
    if isinstance(t, partial): return partial_repr(t)
    if hasattr(t, '__forward_arg__'): return link_type(t.__forward_arg__)
    elif getattr(t, '__args__', None):
        args = t.__args__
        if len(args)==2 and args[1] == type(None):
            return f'`Optional`\[{type_repr(args[0])}\]'
        reprs = ', '.join([type_repr(o) for o in args])
        return f'{link_type(t)}\[{reprs}\]'
    else: return link_type(t)

In [ ]:
def format_param(p):
    "Formats function param to `param1:Type=val`. Font weights: param1=bold, val=bold+italic"
    arg_prefix = arg_prefixes.get(p.kind, '') # asterisk prefix for *args and **kwargs
    res = f"**{arg_prefix}{code_esc(p.name)}**"
    if hasattr(p, 'annotation') and p.annotation != p.empty: res += f':{type_repr(p.annotation)}'
    if p.default != p.empty:
        default = getattr(p.default, 'func', p.default)
        default = getattr(default, '__name__', default)
        res += f'=***`{repr(default)}`***'
    return res

In [ ]:
def format_ft_def(func, full_name:str=None)->str:
    "Format and link `func` definition to show in documentation"
    sig = inspect.signature(func)
    name = f'<code>{full_name or func.__name__}</code>'
    fmt_params = [format_param(param) for name,param
                  in sig.parameters.items() if name not in ('self','cls')]
    arg_str = f"({', '.join(fmt_params)})"
    f_name = f"<code>class</code> {name}" if inspect.isclass(func) else name
    return f'{f_name}',f'{name}{arg_str}'

def get_enum_doc(elt, full_name:str)->str:
    "Formatted enum documentation."
    vals = ', '.join(elt.__members__.keys())
    return f'<code>{full_name}</code>',f'<code>Enum</code> = [{vals}]'

def get_cls_doc(elt, full_name:str)->str:
    "Class definition."
    parent_class = inspect.getclasstree([elt])[-1][0][1][0]
    name,args = format_ft_def(elt, full_name)
    if parent_class != object: args += f' :: {link_type(parent_class, include_bt=True)}'
    return name,args

In [ ]:
def show_doc(elt, doc_string=True, full_name=None, title_level=None, alt_doc_string='', markdown=True):
    "Show documentation for element `elt`. Supported types: class, Callable, and enum."
    anchor_id = get_anchor(elt)
    elt = getattr(elt, '__func__', elt)
    full_name = full_name or fn_name(elt)
    if inspect.isclass(elt):
        if is_enum(elt.__class__):   name,args = get_enum_doc(elt, full_name)
        else:                        name,args = get_cls_doc(elt, full_name)
    elif isinstance(elt, Callable):  name,args = format_ft_def(elt, full_name)
    else: raise Exception(f'doc definition not supported for {full_name}')
    #source_link = get_function_source(elt) if is_fastai_class(elt) else ""
    link = 'https://github.com/fastai/fastai/'
    source_link = '<a href="{link}" class="source_link" style="float:right">[source]</a>'
    title_level = ifnone(title_level, 3 if inspect.isclass(elt) else 4)
    doc =  f'<h{title_level} id="{anchor_id}" class="doc_header">{name}{source_link}</h{title_level}>'
    doc += f'\n\n> {args}\n\n'
    if doc_string and inspect.getdoc(elt):
        doc += inspect.getdoc(elt) + ' '
    if markdown: display(Markdown(doc))
    else: return doc

In [ ]:
show_doc(listify)

<h4 id="listify" class="doc_header"><code>listify</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>listify</code>(**`o`**)

Make `o` a list. 

In [ ]:
show_doc(Pipeline)

<h3 id="Pipeline" class="doc_header"><code>class</code> <code>Pipeline</code><a href="{link}" class="source_link" style="float:right">[source]</a></h3>

> <code>Pipeline</code>(**`tfms`**, **`items`**=***`None`***)

A pipeline of transforms applied to a collection, composed and applied for encode/decode, and setup one at a time 

In [ ]:
show_doc(ConfigKey)

<h3 id="ConfigKey" class="doc_header"><code>ConfigKey</code><a href="{link}" class="source_link" style="float:right">[source]</a></h3>

> <code>Enum</code> = [Data, Archive, Model]

Keys for a path in the fastai config file 

In [ ]:
show_doc(compose)

<h4 id="compose" class="doc_header"><code>compose</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>compose</code>(**\*`funcs`**:`Callable`)

Create a function that composes all functions in `funcs`, passing along remaining `*args` and `**kwargs` to all 

In [ ]:
show_doc(untar_data)

<h4 id="untar_data" class="doc_header"><code>untar_data</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>untar_data</code>(**`url`**, **`fname`**=***`None`***, **`dest`**=***`None`***, **`c_key`**=***`<ConfigKey.Data: 1>`***, **`force_download`**=***`False`***)

Download `url` to `fname` if `dest` doesn't exist, and un-tgz to folder `dest`. 

In [ ]:
show_doc(add_docs)

<h4 id="add_docs" class="doc_header"><code>add_docs</code><a href="{link}" class="source_link" style="float:right">[source]</a></h4>

> <code>add_docs</code>(**\*\*`docs`**)

Copy values from `docs` to `cls` docstrings, and confirm all public methods are documented 

In [ ]:
doc =  f'<h{title_level} id="{anchor_id}" class="doc_header">{name}{source_link}{test_link}</h{title_level}>'
    doc += f'\n\n> {args}\n\n'
    doc += f'{test_modal}'